# AI in Biomedicine and Healthcare [BM5020]
# Project : Vessel and tissue recognition during third-space Endoscopy using a deep learning algorithm
## Team Members
- **Pradeep Mundlik, ai21btech11022@iith.ac.in**
- **Naman Chhibbar, ma21btech110xx@iith.ac.in**

#### Link to Github Repository: [Github](https://github.com/NamanChhibbar/BM5020-Project)
#### Link to Paper PPT: [Slides](https://docs.google.com/presentation/d/1uuP4rpO48ZUkxz7x2UBRvn2n522-tvPUc-YlY8JwQa8/edit?usp=sharing)


***This notebook contains the main code for the project. It includes the implementation of various algorithms and functions related to the project along with output to help with our analysis.***

In [49]:
# importing resnest model from timm library
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)

('dog.jpg', <http.client.HTTPMessage at 0x792ab277fa00>)

In [50]:
# defining the model
class Resnest(nn.Module):
    def __init__(self, model_name='resnest101e', pretrained=True):
        super(Resnest, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        # self.model.fc = nn.Linear(n_features, 10)
        self.model.fc = nn.Identity(n_features)

    def forward(self, x):
        x = self.model(x)
        return x
    
# define the preprocessing function
def preprocess(image):
    image = image.convert('RGB')
    image = np.array(image)
    image = np.transpose(image, (2, 0, 1))
    image = torch.tensor(image, dtype=torch.float32)
    image /= 255.0
    normalise = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    image = normalise(image)
    return image

In [54]:
# load model
model = Resnest()
# print(model)
model.eval()

# load image
image = Image.open('dog.jpg')
image = preprocess(image)

# predict
with torch.no_grad():
    pred = model(image)
    # pred = torch.sigmoid(pred)
    pred = pred.numpy()

pred = np.squeeze(pred)
print(pred)

[0.01111571 0.01799265 0.03249104 ... 0.01077665 0.08971523 1.6484196 ]


In [66]:
# import deep lab v3 model

!pip3 install torchsummary
!pip3 install torchviz

from torchvision import models
from torchsummary import summary
from torchviz import make_dot
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

# define the model
model = models.segmentation.DeepLabV3()
print(model)
print(summary(model, input_size=(3, 224, 224)))

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


TypeError: _SimpleSegmentationModel.__init__() missing 2 required positional arguments: 'backbone' and 'classifier'